In [27]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from functools import reduce
from operator import mul
import shamirs



# XOR-based Secret Sharing
def encode(secret):
    M = bin(secret)[2:]
    if len(M) % 2 != 0:
        M += '0'
    half_len = len(M) // 2
    M1 = M[:half_len]
    M2 = M[half_len:]
    R1 = [random.randint(0, 1) for _ in range(half_len)]
    R2 = [random.randint(0, 1) for _ in range(half_len)]
    E1 = R1 + [int(M2[i]) ^ R2[i] for i in range(half_len)]
    E2 = [int(M1[i]) ^ R1[i] for i in range(half_len)] + R2
    E3 = [int(M1[i]) ^ R2[i] for i in range(half_len)] + [int(M2[i]) ^ R1[i] for i in range(half_len)]
    return E1, E2, E3

def decode(part1, part2, E1, E2, E3):
    half_len = len(part1) // 2
    if part1 == E1 and part2 == E2:
        R1 = part1[:half_len]
        R2 = part2[half_len:]
        M1 = [part2[i] ^ R1[i] for i in range(half_len)]
        M2 = [part1[half_len + i] ^ R2[i] for i in range(half_len)]
    elif part1 == E1 and part2 == E3:
        R1 = part1[:half_len]
        M2 = [part2[half_len + i] ^ R1[i] for i in range(half_len)]
        R2 = [part1[half_len + i] ^ M2[i] for i in range(half_len)]
        M1 = [part2[i] ^ R2[i] for i in range(half_len)]
    elif part1 == E2 and part2 == E3:
        R2 = part1[half_len:]
        M1 = [part2[i] ^ R2[i] for i in range(half_len)]
        R1 = [part1[i] ^ M1[i] for i in range(half_len)]
        M2 = [part2[half_len + i] ^ R1[i] for i in range(half_len)]
    M = M1 + M2
    return int(''.join(map(str, M)), 2)

# Number of iterations
iterations = 10000

# Arrays to store runtimess
encoder_runtimes_shamir = []
decoder_runtimes_shamir = []
encoder_runtimes_xor = []
decoder_runtimes_E1_E2 = []
decoder_runtimes_E1_E3 = []
decoder_runtimes_E2_E3 = []

secret = 12345
n = 3
k = 2

for _ in range(iterations):
    # Shamir's Scheme
    start_time = time.time()
    shares = shamirs.shares(secret, quantity=3, threshold=2)
    encoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to milliseconds

    start_time = time.time()
    recovered_secret =shamirs.interpolate(shares)
    decoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to milliseconds

    # XOR-based Secret Sharing
    start_time = time.time()
    E1, E2, E3 = encode(secret)
    encoder_runtimes_xor.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    decode(E1, E2, E1, E2, E3)
    decoder_runtimes_E1_E2.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    decode(E1, E3, E1, E2, E3)
    decoder_runtimes_E1_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    decode(E2, E3, E1, E2, E3)
    decoder_runtimes_E2_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

# Calculating the averages
avg_encoder_runtime_shamir = np.mean(encoder_runtimes_shamir)
print(avg_encoder_runtime_shamir)
avg_decoder_runtime_shamir = np.mean(decoder_runtimes_shamir)
print(avg_decoder_runtime_shamir)
avg_encoder_runtime_xor = np.mean(encoder_runtimes_xor)
print(avg_encoder_runtime_xor)
avg_decoder_runtime_E1_E2 = np.mean(decoder_runtimes_E1_E2)
print(avg_decoder_runtime_E1_E2)
avg_decoder_runtime_E1_E3 = np.mean(decoder_runtimes_E1_E3)
print(avg_decoder_runtime_E1_E3)
avg_decoder_runtime_E2_E3 = np.mean(decoder_runtimes_E2_E3)
print(avg_decoder_runtime_E2_E3)

# Plotting the results
plt.figure(figsize=(14, 7), dpi=100)

plt.subplot(1, 2, 1)
plt.bar(['Shamir', 'XOR'], [avg_encoder_runtime_shamir, avg_encoder_runtime_xor], color=['#1f77b4', '#ff7f0e'])
plt.title('Average Encoding Runtime (2, 3)', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.tight_layout()
plt.show()


Shamir's Secret Sharing:
Encoder Runtime: 0.000259 seconds
Decoder Runtime: 0.000017 seconds
Recovered Secret: 12345

Shares: E1=[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1], E2=[1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0], E3=[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0]
Encoder Runtime: 0.000030 seconds
Recovered Secret (E1 and E2): 12345
Decoder Runtime: 0.000012 seconds
Recovered Secret (E1 and E3): 12345
Decoder Runtime: 0.000006 seconds
Recovered Secret (E2 and E3): 12345
Decoder Runtime: 0.000005 seconds


In [ ]:
# Plotting the results
plt.figure(figsize=(14, 7), dpi=100)

plt.subplot(1, 2, 2)
plt.bar(['Shamir', 'E1,E2', 'E1,E3', 'E2,E3'],
        [avg_decoder_runtime_shamir, avg_decoder_runtime_E1_E2, avg_decoder_runtime_E1_E3, avg_decoder_runtime_E2_E3],
        color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Average Decoding Runtime (2, 3)', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.tight_layout()
plt.show()